In [1]:
import pandas as pd

#Imports the cleaned questions
df = pd.read_csv('CleanedQuestions.csv')
df = df.loc[:, ~df.columns.str.contains('Unnamed')]

In [1]:
from flask import *
import json

#creates flask app
app = Flask(__name__)

#creates API route for the main page which just confirms that the API has loaded correctly
@app.route('/', methods=['GET'])
def home_page():
    data_set = {'Message': 'Successfully loaded API'}
    json_dump = json.dumps(data_set)

    return json_dump

#Creates API route for question page 
@app.route('/question/', methods=['GET']) 
def question_request():
    #gets the user's question from the api http request
    question = str(request.args.get('question')) #/question/?question=question
    #sets the return data message to the question so the user can see its registered the question input
    data_set = {'Message': f'{question}'}
    json_dump = json.dumps(data_set)
    
    #uses rapid fuzz to match the user question with question's in the database and then extracts the name (link to the question post) from the question in the database
    from rapidfuzz import process, fuzz

    most_similar = process.extractOne(question, df['Question'], scorer=fuzz.WRatio)
    most_similar = list(most_similar)


    questionName = df.loc[df['Question'] == most_similar[0]]['Name']
    questionName = list(questionName)[0]



    #Accesses the reddit api and makes sure it's working
    CLIENT_ID = 'lpakYt45RJlvK6Lp6ybehw'
    SECRET_KEY = 'XXHGvxgxAhlEHK6-KMy8nP_QmcedXg'

    import requests

    auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_KEY)
    data = {
        'grant_type' : 'password',
        'username' : 'PiccoloNo3695',
        'password' : 'scooby2203'
    }

    headers = {'User-Agent' : 'MyAPI/0.0.1'}
    res = requests.post('https://www.reddit.com/api/v1/access_token', auth=auth, data=data, headers=headers)
    TOKEN = res.json()['access_token']
    headers['Authorization'] = f'bearer {TOKEN}'

    requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)



    #makes a link variable that is equal to the api link for getting comments from a specific post by adding the name of the selected post to the link and then makes a get request 
    #to fetch all the comments from the selected post
    link = 'http://oauth.reddit.com/r/python/comments/' + str(questionName)
    res = requests.get(link, headers=headers, params={'limit' : '100'})
    res.json()[0]



    #Converts the comment json into a readable dataframe format with just the important information that been the comment text and the score of the comment
    import pandas as pd

    df2 = pd.DataFrame()
    for post in res.json()[1]['data']['children']:
        df2 = df2.append({
            'Awnser' : post ['data']['body'],
            'Score' : post ['data']['score']
        }, ignore_index=True)



    #Makes the awnser variable equal to the awnser with the highest score
    awnser = df2.loc[df2['Score'].idxmax()]
    awnser = awnser['Awnser']

    #creates a route to send the awnser back to the user
    @app.route('/awnser/', methods=['GET']) 
    def awnser_():
        data_set = {'Message': f'{awnser}'}
        json_dump = json.dumps(data_set)

        return json_dump

    

    return json_dump

#runs the api on port 7777 on the local machine
if __name__ == '__main__':
    app.run(port=7777)



    

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:7777 (Press CTRL+C to quit)
